In [1]:
# Warning control
import warnings

warnings.filterwarnings("ignore")

import os
import io
import IPython.display
from PIL import Image
import base64

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv() # read local .env file

from huggingface_hub import login

login(os.environ['HF_SK'])

#Checking the access to the openai secret key
HF_SK = os.getenv("HF_SK")
print("HuggingFace Hub API Token:", HF_SK[:8] + "..." if HF_SK else "Key not found")

HuggingFace Hub API Token: hf_NawcY...


In [2]:
import pandas as pd

suppliers_data = {
    "name": [
        "Montreal Ice Cream Co",
        "Brain Freeze Brothers",
        "Toronto Gelato Ltd",
        "Buffalo Scoops",
        "Vermont Creamery",
    ],
    "location": [
        "Montreal, QC",
        "Burlington, VT",
        "Toronto, ON",
        "Buffalo, NY",
        "Portland, ME",
    ],
    "distance_km": [120, 85, 400, 220, 280],
    "canadian": [True, False, True, False, False],
    "price_per_liter": [1.95, 1.91, 1.82, 2.43, 2.33],
    "tasting_fee": [0, 12.50, 30.14, 42.00, 0.20],
}

data_description = """Suppliers have an additional tasting fee: that is a fixed fee applied to each order to taste the ice cream."""
suppliers_df = pd.DataFrame(suppliers_data)
suppliers_df

,name,location,distance_km,canadian,price_per_liter,tasting_fee
0,Montreal Ice Cream Co,"Montreal, QC",120,True,1.95,0.00
1,Brain Freeze Brothers,"Burlington, VT",85,False,1.91,12.50
2,Toronto Gelato Ltd,"Toronto, ON",400,True,1.82,30.14
3,Buffalo Scoops,"Buffalo, NY",220,False,2.43,42.00
4,Vermont Creamery,"Portland, ME",280,False,2.33,0.20


**Setup tools**

In [4]:
import numpy as np

def calculate_daily_supplier_price(row):
    order_volume = 30

    # Calculate raw product cost
    product_cost = row["price_per_liter"] * order_volume

    # Calculate transport cost
    trucks_needed = np.ceil(order_volume / 300)
    cost_per_km = 1.20
    transport_cost = row["distance_km"] * cost_per_km * trucks_needed

    # Calculate tariffs for ice cream imported from Canada
    tariff = product_cost * np.pi / 50 * row["canadian"]

    # Get total cost
    total_cost = product_cost + transport_cost + tariff + row["tasting_fee"]
    return total_cost

suppliers_df["daily_price"] = suppliers_df.apply(calculate_daily_supplier_price, axis=1)
display(suppliers_df)

# Let's remove this column now.
suppliers_df = suppliers_df.drop("daily_price", axis=1)

,name,location,distance_km,canadian,price_per_liter,tasting_fee,daily_price
0,Montreal Ice Cream Co,"Montreal, QC",120,True,1.95,0.00,206.175663
1,Brain Freeze Brothers,"Burlington, VT",85,False,1.91,12.50,171.800000
2,Toronto Gelato Ltd,"Toronto, ON",400,True,1.82,30.14,568.170619
3,Buffalo Scoops,"Buffalo, NY",220,False,2.43,42.00,378.900000
4,Vermont Creamery,"Portland, ME",280,False,2.33,0.20,406.100000


In [5]:
# First we make a few tools
from smolagents import tool

@tool
def calculate_transport_cost(distance_km: float, order_volume: float) -> float:
    """
    Calculate transportation cost based on distance and order size.
    Refrigerated transport costs $1.2 per kilometer and has a capacity of 300 liters.

    Args:
        distance_km: the distance in kilometers
        order_volume: the order volume in liters
    """
    trucks_needed = np.ceil(order_volume / 300)
    cost_per_km = 1.20
    return distance_km * cost_per_km * trucks_needed


@tool
def calculate_tariff(base_cost: float, is_canadian: bool) -> float:
    """
    Calculates tariff for Canadian imports. Returns the tariff only, not the total cost.
    Assumes tariff on dairy products from Canada is worth 2 * pi / 100, approx 6.2%

    Args:
        base_cost: the base cost of goods, not including transportation cost.
        is_canadian: wether the import is from Canada.
    """
    if is_canadian:
        return base_cost * np.pi / 50
    return 0

The `tool` methods should MUST have `description` and `Args` details. The agents will use this information for its usage. 

In [6]:
calculate_transport_cost.description

'Calculate transportation cost based on distance and order size.\nRefrigerated transport costs $1.2 per kilometer and has a capacity of 300 liters.'

**Setup model**

In [7]:
from smolagents import HfApiModel, CodeAgent
# from helper import get_huggingface_token

model = HfApiModel(
    "Qwen/Qwen2.5-72B-Instruct",
    provider="together", # Choose a specific inference provider, here is TogetherAI
    max_tokens=4096,
    temperature=0.1
)

**Setup the Code Agent**

In [8]:
agent = CodeAgent(
    model=model,
    tools=[calculate_transport_cost, calculate_tariff],
    max_steps=10,
    additional_authorized_imports=["pandas", "numpy"],
    verbosity_level=2
)
agent.logger.console.width=66

In [10]:
agent.run(
    """Can you get me the transportation cost for 50 liters
    of ice cream over 10 kilometers?"""
)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Can you get me the transportation cost for 50 liters           │
│     of ice cream over 10 kilometers?                           │
│                                                                │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ───────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ───────────────────────────────────────
Thought: I will use the `calculate_transport_cost` function to    
calculate the transportation cost for 50 liters of ice cream over 
10 kilometers.                                                    
Code:                                                             
```py                                                             
distance_km = 10                                                  
order_volume = 50                                                 
transport_cost = calculate_transport_cost(distance_km,            
order_volume)                                                     
print(f"The transportation cost for 50 liters of ice cream over 10
kilometers is ${transport_cost:.2f}.")                            
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  distance_km = 10                                                
  order_volume = 50                                               
  transport_cost = calculate_transport_cost(distance_km,          
  order_volume)                                                   
  print(f"The transportation cost for 50 liters of ice cream      
  over 10 kilometers is ${transport_cost:.2f}.")                  
 ────────────────────────────────────────────────────────────────

Execution logs:
The transportation cost for 50 liters of ice cream over 10 
kilometers is $12.00.

Out: None

...

Output message of the LLM: ───────────────────────────────────────
Thought: I have the transportation cost for 50 liters of ice cream
over 10 kilometers. I will now provide the final answer.          
Code:                                                             
```py                                                             
final_answer(12.00)                                               
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  final_answer(12.00)                                             
 ────────────────────────────────────────────────────────────────

Out - Final answer: 12.0

[Step 2: Duration 1.34 seconds| Input tokens: 4,608 | Output 
tokens: 142]

12.0